# Setup for data collection



## Packages needed


In [ ]:
from datetime import datetime, timedelta
from shapely.geometry import Polygon, Point
import numpy as np
import requests
import pandas as pd
from shapely.geometry import Polygon
from xml.etree import ElementTree as ET
from shapely.geometry import Polygon
import os

## Functions needed

In [ ]:
#Function to create an api request
def make_api_request(url, method="GET", data=None, headers=None):
    global access_token
    if not headers:
        headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.request(method, url, json=data, headers=headers)
    if response.status_code in [401, 403]:
        global refresh_token
        access_token = refresh_access_token(refresh_token)
        headers["Authorization"] = f"Bearer {access_token}"
        response = requests.request(method, url, json=data, headers=headers)
    return response

#Data collection function bassed on start and end date
def query_sentinel2_data(start_date, end_date, token):
    """
    Queries Sentinel-2 data within a specified time range from the Copernicus Data Space,
    targeting data collected over the Swiss Alps.

    Parameters:
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    token (str): Access token for authentication.

    Returns:
    DataFrame: Contains details about the Sentinel-2 images.
    """
    all_data = []
    #Polygon are selected as areo of interest
    saas_fee_polygon = (
    "POLYGON ((7.562714 46.244451, 7.419891 45.823057, 8.260345 45.850804, "
    "8.175201 46.320378, 7.562714 46.244451))"
)



    # Adjust filter string for Sentinel-2
    filter_string = (
        f"Collection/Name eq 'SENTINEL-2' and "
        f"Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/Value eq 'S2MSI2A') and "
        f"ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T23:59:59.999Z"
    )

    next_url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter={filter_string} and "
        f"OData.CSC.Intersects(area=geography'SRID=4326;{saas_fee_polygon}')&"
        f"$top=1000"
    )

    headers = {"Authorization": f"Bearer {token}"}

    while next_url:
        response = make_api_request(next_url, headers=headers)
        if response.status_code == 200:
            data = response.json()["value"]
            all_data.extend(data)
            next_url = response.json().get("@odata.nextLink")
        else:
            print(f"Error fetching data: {response.status_code} - {response.text}")
            break

    return pd.DataFrame(all_data)


#Password credentials connection
def get_access_and_refresh_token(username, password):
    """Retrieve both access and refresh tokens."""
    url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    data = {
        "grant_type": "password",
        "username": username,
        "password": password,
        "client_id": "cdse-public",
    }
    response = requests.post(url, data=data)
    response.raise_for_status()
    tokens = response.json()
    return tokens["access_token"], tokens["refresh_token"]

#Access token function
def refresh_access_token(refresh_token):
    """Attempt to refresh the access token using the refresh token."""
    url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
        "client_id": "cdse-public",
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    try:
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()  # This will throw an error for non-2xx responses
        return response.json()["access_token"]
    except requests.exceptions.HTTPError as e:
        print(f"Failed to refresh token: {e.response.status_code} - {e.response.text}")
        if e.response.status_code == 400:
            print("Refresh token invalid, attempting re-authentication...")
            # Attempt to re-authenticate
            username = username
            password = password
            # This requires securely managing the credentials, which might not be feasible in all contexts
            access_token, new_refresh_token = get_access_and_refresh_token(
                username, password
            )  # This is a placeholder
            refresh_token = (
                new_refresh_token  # Update the global refresh token with the new one
            )
            return access_token
        else:
            raise

def download_single_product(
    product_id, file_name, access_token, download_dir="downloaded_products"
):
    """
    Download a single product from the Copernicus Data Space.

    :param product_id: The unique identifier for the product.
    :param file_name: The name of the file to be downloaded.
    :param access_token: The access token for authorization.
    :param download_dir: The directory where the product will be saved.
    """
    # Ensure the download directory exists
    os.makedirs(download_dir, exist_ok=True)

    # Construct the download URL
    url = (
        f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
    )

    # Set up the session and headers
    headers = {"Authorization": f"Bearer {access_token}"}
    session = requests.Session()
    session.headers.update(headers)

    # Perform the request
    response = session.get(url, headers=headers, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Define the path for the output file
        output_file_path = os.path.join(download_dir, file_name + ".zip")

        # Stream the content to a file
        with open(output_file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded: {output_file_path}")
    else:
        print(
            f"Failed to download product {product_id}. Status Code: {response.status_code}"
        )

## Extract data

In [ ]:
username = "" #Insert your username
password = "" #Insert your password
access_token, refresh_token = get_access_and_refresh_token(username, password)
start_date = "2023-07-20" #Desired start date
end_date = "2023-07-20" #Desired end date

sentinel2_data = query_sentinel2_data(
    start_date, end_date, access_token
)

In [ ]:
from IPython.display import display

display(sentinel2_data)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,11620cc7-b808-4071-83db-8ccedf2559f0,S2B_MSIL2A_20230720T101609_N0509_R065_T32TMS_2...,application/octet-stream,1234476014,2023-07-20T16:05:21.824000Z,2023-07-20T16:23:22.646220Z,2023-07-20T17:06:41.321769Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2023/07/20/S2B_MSIL...,"[{'Value': '9e2769061f1a2628346dc048d38b823b',...","{'Start': '2023-07-20T10:16:09.030000Z', 'End'...",geography'SRID=4326;POLYGON ((7.6911307723828 ...,"{'type': 'Polygon', 'coordinates': [[[7.691130..."
1,application/octet-stream,a644fff5-9930-4c9d-90c2-58656f7cbad8,S2B_MSIL2A_20230720T101609_N0509_R065_T32TMR_2...,application/octet-stream,1207665840,2023-07-20T15:56:01.257000Z,2023-07-20T16:08:51.311341Z,2023-07-20T17:03:20.863994Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2023/07/20/S2B_MSIL...,"[{'Value': '4ae4b90294e8cc85d4e5f54b16083cab',...","{'Start': '2023-07-20T10:16:09.030000Z', 'End'...",geography'SRID=4326;POLYGON ((7.70695108472353...,"{'type': 'Polygon', 'coordinates': [[[7.706951..."
2,application/octet-stream,b48c143e-a482-4010-966b-9af220fb0f44,S2B_MSIL2A_20230720T101609_N0509_R065_T32TLS_2...,application/octet-stream,241762516,2023-07-20T15:56:12.803000Z,2023-07-20T16:02:43.491421Z,2023-07-20T17:01:43.461950Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2023/07/20/S2B_MSIL...,"[{'Value': '7e4f2e982fa0ff215bd785dd389356d3',...","{'Start': '2023-07-20T10:16:09.030000Z', 'End'...",geography'SRID=4326;POLYGON ((7.43196905385645...,"{'type': 'Polygon', 'coordinates': [[[7.431969..."
3,application/octet-stream,b2f8c6b2-9f3c-4e6d-8e08-62761e2bb5ca,S2B_MSIL2A_20230720T101609_N0509_R065_T32TLR_2...,application/octet-stream,560197543,2023-07-20T16:03:04.163000Z,2023-07-20T16:10:01.095426Z,2023-07-20T17:03:26.740626Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2023/07/20/S2B_MSIL...,"[{'Value': '839fc9f01b655c42e5c7dd2b0efc8e75',...","{'Start': '2023-07-20T10:16:09.030000Z', 'End'...",geography'SRID=4326;POLYGON ((7.11843280154279...,"{'type': 'Polygon', 'coordinates': [[[7.118432..."


# Download data

In [ ]:
def query_product_by_name(product_name, token):
    """
    Query a specific Sentinel-2 product by its name.

    Parameters:
    product_name (str): The exact name of the product to search for.
    token (str): Access token for authentication.

    Returns:
    dict: Metadata for the matching product.
    """
    url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter=Name eq '{product_name}'"
    )
    headers = {"Authorization": f"Bearer {token}"}

    response = make_api_request(url, headers=headers)
    if response.status_code == 200:
        data = response.json().get("value", [])
        if data:
            return data[0]  # Return the first matching product (if any)
        else:
            print(f"No product found with name: {product_name}")
            return None
    else:
        print(f"Error fetching product: {response.status_code} - {response.text}")
        return None



username = "" #Insert username
password = "" #Insert Password

# Step 1: Authenticate and retrieve tokens
access_token, refresh_token = get_access_and_refresh_token(username, password)

# Step 2: Provide the product name
product_name = "S2B_MSIL2A_20230720T101609_N0509_R065_T32TMS_20230720T131906.SAFE"  # Replace with the specific product name you have

# Step 3: Query the product by name
product_metadata = query_product_by_name(product_name, access_token)

if product_metadata:
    product_id = product_metadata["Id"]  # Extract product ID from metadata
    file_name = product_metadata["Name"]  # Extract product name from metadata

    # Step 4: Download the product
    download_dir = ""  # Replace with your desired directory
    download_single_product(product_id, file_name, access_token, download_dir)

Downloaded: /content/drive/MyDrive/GEOL0069_data/Final Project/S2B_MSIL2A_20230720T101609_N0509_R065_T32TMS_20230720T131906.SAFE.zip
